In [1]:
import os

To make sure the notebook can run smoothly, do the following:
1. Install all the required packages. However, if you are prompted to install "Crypto", DO NOT do "pip install Crypto"! DO "pip install pycryptodome" instead. Here are more details: https://medium.com/@rajparmar23801/no-module-named-crypto-707f75186a6a
2. Go to Firebase. Click the gear icon next to "Project Overview". Choose "Project Settings" - Service Accounts - Generate New Private Key. Put the generated json file in the same directory as this notebook, and change the "serviceAccount" attribute in the next code block to the name of the json file you just downloaded.

## Firebase Storage Download (Image & Audio Data)

In [3]:
from firebase import Firebase

config = {
  "apiKey": "8YOlQyD5MNnZlMl4hAvqYz3uePYTO9bmuS7s9mAo",
  "authDomain": "capstone-caf92.firebaseapp.com",
  "databaseURL": "https://databaseName.firebaseio.com",
  "storageBucket": "capstone-caf92.appspot.com",
  "serviceAccount": "capstone-caf92-firebase-adminsdk-uckm2-XXXXXX.json"
}

firebase = Firebase(config)

In [4]:
storage = firebase.storage()

In [17]:
all_files = storage.list_files()

for file in all_files:
    directory = file.name.split("/")
    target_dir = "data/" + "/".join(directory[:-1]) + "/"
    if target_dir == "data/Feb8-1/images/":
        print(file.name)
        isExist = os.path.exists(target_dir)
        if isExist:
            if os.path.isfile("data/" + file.name):
                continue
        else:
            os.makedirs(target_dir)
        storage.child(file.name).download("data/" + file.name)

Feb8-1/images/2023-02-08-15-49-02.jpeg
Feb8-1/images/2023-02-08-15-49-03.jpeg
Feb8-1/images/2023-02-08-15-49-04.jpeg
Feb8-1/images/2023-02-08-15-49-05.jpeg
Feb8-1/images/2023-02-08-15-49-06.jpeg
Feb8-1/images/2023-02-08-15-49-07.jpeg
Feb8-1/images/2023-02-08-15-49-08.jpeg
Feb8-1/images/2023-02-08-15-49-09.jpeg
Feb8-1/images/2023-02-08-15-49-10.jpeg
Feb8-1/images/2023-02-08-15-49-11.jpeg
Feb8-1/images/2023-02-08-15-49-12.jpeg
Feb8-1/images/2023-02-08-15-49-13.jpeg
Feb8-1/images/2023-02-08-15-49-14.jpeg
Feb8-1/images/2023-02-08-15-49-15.jpeg
Feb8-1/images/2023-02-08-15-49-16.jpeg
Feb8-1/images/2023-02-08-15-49-17.jpeg
Feb8-1/images/2023-02-08-15-49-18.jpeg
Feb8-1/images/2023-02-08-15-49-19.jpeg
Feb8-1/images/2023-02-08-15-49-20.jpeg
Feb8-1/images/2023-02-08-15-49-21.jpeg
Feb8-1/images/2023-02-08-15-49-22.jpeg
Feb8-1/images/2023-02-08-15-49-24.jpeg
Feb8-1/images/2023-02-08-15-49-25.jpeg
Feb8-1/images/2023-02-08-15-49-26.jpeg
Feb8-1/images/2023-02-08-15-49-27.jpeg
Feb8-1/images/2023-02-08-

## Firestore Download (Labels & Other Text Data)

In [6]:
import firebase_admin
import csv
from firebase_admin import credentials, firestore

ModuleNotFoundError: No module named 'firebase_admin'

In [8]:
cred = credentials.Certificate("capstone-caf92-firebase-adminsdk-uckm2-7262153063.json")
firebase_admin.initialize_app(cred)

### List current collection names

In [5]:
db = firestore.client()
listCollections = []
for col in db.collections():
    listCollections.append(col)
    print(col.id)

NameError: name 'firestore' is not defined

### Pull the target collection to CSV (change target_id)

In [19]:
target_id = "Feb2" #Change this to the collection you want (to limit read and prevent exceeding quota)
for col in listCollections:
    if col.id != target_id:
        continue
    #Check if target directory & file exists
    target_dir = "data/firestore/"
    isExist = os.path.exists(target_dir)
    #Audio part
    if isExist:
        if os.path.isfile(target_dir + col.id + "_audio_raw.csv") or os.path.isfile(target_dir + col.id + "_audio_labels.csv"):
            continue
    else:
        os.makedirs(target_dir)
    audio = col.document('audio').collections()
    for entry in audio:
        audioList = []
        for doc in entry.stream():
            audioList.append(doc.to_dict())
        field_names = list(audioList[0].keys())
        with open(target_dir + col.id + "_audio_" + entry.id + ".csv","w") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames = field_names)
            writer.writeheader()
            writer.writerows(audioList)
    #Image part
    if isExist:
        if os.path.isfile(target_dir + col.id + "_images_thermal.csv"):
            continue
    else:
        os.makedirs(target_dir)
    image = col.document('images').collections()
    for entry in audio:
        imageList = []
        for doc in entry.stream():
            imageList.append(doc.to_dict())
        field_names = list(imageList[0].keys())
        with open(target_dir + col.id + "_images_" + entry.id + ".csv","w") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames = field_names)
            writer.writeheader()
            writer.writerows(imageList)
